# 📊 Análise de Resultados — PySpark + Delta Lake

Este notebook tem como objetivo realizar análises exploratórias e agregações a partir dos dados estruturados na camada **Trusted** do Data Warehouse, utilizando **PySpark** em conjunto com o **Delta Lake**. Serão exploradas informações provenientes de tabelas fato e dimensões, permitindo insights sobre o perfil das empresas, distribuição geográfica, porte, natureza jurídica, entre outros aspectos relevantes.

## Configurações e importes

In [158]:
# Para iniciar a seção spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DecimalType
from pyspark.sql import functions as f
from datetime import datetime
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [159]:
builder = SparkSession.builder \
    .appName("App analise de dados") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

spark

## MERGE

Tabelas para merge: 

- cnpj_base & cnpj_basico & empresa_cnpj_basico
- codigo_municipio & id_municipio
- cod_natureza_juridica & codigo_natureza_juridica

Iniciar deltaTable

In [ ]:
try:    
    deltaTable: DeltaTable = DeltaTable.forPath(spark, "../TRS/CONSOLIDADO/01_TABELA_FATO")
    df: DataFrame = deltaTable.toDF()
    df.show(5, truncate=False)
    df.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+--------------+---------+----------+-------+-------------+------------------------------------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+---------------------------------------------------------------------------------------------------------------+---------------+-----------------------------+------+---------------+--------+--------+---+----------------+----+---------+----+---------+-------+--------+-------------------------+-----------------+----------------------+-----------+---------------------------------------------------------------------------------------+------------+----------------+-------------------+------------+---------------------+-------------------------------+--------------+--------------------+---------------------------+------------------------+-----------------+-----------+-------------+------------------+---------------------+---------+--------------+------------

In [123]:
# Realizando um UPINSERT
try:
    deltaTable.alias("target") \
        .merge(
            df.alias("source"),
            condition="target.cnpj_completo = source.cnpj_completo"
    ) \
        .whenMatchedUpdate(
            set={
                "cnpj_completo": "source.cnpj_completo",
                "cod_natureza_juridica": "source.codigo_natureza_juridica",
            }
    ).whenNotMatchedInsert(
            values={
                "cnpj_completo": "source.cnpj_completo",
                "cod_natureza_juridica": "source.codigo_natureza_juridica",
            }
    ).execute()

    print("Merge efetuado!")
except Exception as e:
    print(f"Erro ao efetuar merge: {e}")

Merge efetuado!


In [ ]:
def atualizar_delta_table():
    try:
        df.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .partitionBy("uf") \
            .save("../CONSOLIDADO/01_TABELA_FATO")
        print("Dados gravados!")
    except Exception as e:
        print(f"Erro ao gravar novos dados: {e}")

# atualizar_delta_table()

## 📊 Análise de Resultados

### Iniciar DeltaSpark

In [152]:
try:    
    deltaTable: DeltaTable = DeltaTable.forPath(spark, "../TRS/CONSOLIDADO/01_TABELA_FATO")
    df: DataFrame = deltaTable.toDF()
    df.show(5, truncate=False)
    df.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

+--------------+---------+----------+-------+-------------+---------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+-----------------------------------------------+---------------+-------------------------------+------+--------------------------+-----------------------+--------+---+----------------+----+---------+----+---------+-------+----+----------------------------+-----------------+----------------------+-----------+--------------------------------------------------------------------------------------------+------------+-----------+-------------------+------------+---------------------+-------------------------------+--------------+--------------------+---------------------------+------------------------+-----------------+-----------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_completo |cnpj_base|cnpj_ordem|cnpj_dv|matriz_filia

### Visão Geral

In [157]:
df.select(
    df["cnpj_completo"].alias("CNPJ"),
    df["nome_fantasia"].alias("NOME FANTASIA"),
    df["situacao_cadastral"].alias("SITUAÇÃO CADASTRAL"),
    f.date_format(df["data_situacao_cadastral"], "dd/MM/yyyy").alias("DATA SITUAÇÃO CADASTRAL"),
    df["motivo_situacao_cadastral"].alias("SITUAÇÃO CADASTRAL MOTIVO"),
    df["nome_cidade_exterior"].alias("CIDADE EXTERIOR"),
    df["codigo_pais"].alias("CÓDIGO DO PAÍS"),
    f.date_format(df["data_inicio_atividade"], "dd/MM/yyyy").alias("DATA INÍCIO ATIVIDADE"),
    df["cnae_fiscal"].alias("CNAE PRINCIPAL"),
    df["cnae_fiscal_secundaria"].alias("CNAES SECUNDÁRIOS"),
    f.concat(
        df["tipo_logradouro"],
        f.lit(" "),
        df["logradouro"],
        f.lit(", "),
        df["numero"]
    ).alias("ENDEREÇO"),
    df["complemento"].alias("COMPLEMENTO"),
    df["bairro"].alias("BAIRRO"),
    df["cep"].alias("CEP"),
    df["municipio"].alias("MUNICÍPIO"),
    df["uf"].alias("UF"),
    df["telefone1"].alias("TELEFONE"),
    df["email"].alias("EMAIL"),
    df["descricao_cnae"].alias("DESCRIÇÃO CNAE"),
    df["razao_social"].alias("RAZÃO SOCIAL"),
    df["natureza_juridica"].alias("NATUREZA JURÍDICA"),
    df["codigo_porte_empresa"].alias("CÓDIGO PORTE"),
    df["opcao_simples"].alias("OPÇÃO PELO SIMPLES"),
    f.date_format(df["data_opcao_simples"], "dd/MM/yyyy").alias("DATA OPÇÃO SIMPLES"),
    f.date_format(df["data_exclusao_simples"], "dd/MM/yyyy").alias("DATA EXCLUSÃO SIMPLES"),
    df["opcao_mei"].alias("OPÇÃO MEI"),
    f.date_format(df["data_opcao_mei"], "dd/MM/yyyy").alias("DATA OPÇÃO MEI"),
    f.date_format(df["data_exclusao_mei"], "dd/MM/yyyy").alias("DATA EXCLUSÃO MEI")
).filter(df["nome_fantasia"].like("%BANCO DO BRASIL%")).show(10, truncate=False)

+--------------+----------------------------------------------+------------------+-----------------------+-------------------------+---------------+--------------+---------------------+--------------+-----------------+----------------------------------+-------------------+--------------+--------+--------------+---+--------+------------------------------+------------------------------------------------------------------------------------------------------+----------------------------------------------------------+-----------------------+------------+------------------+------------------+---------------------+---------+--------------+-----------------+
|CNPJ          |NOME FANTASIA                                 |SITUAÇÃO CADASTRAL|DATA SITUAÇÃO CADASTRAL|SITUAÇÃO CADASTRAL MOTIVO|CIDADE EXTERIOR|CÓDIGO DO PAÍS|DATA INÍCIO ATIVIDADE|CNAE PRINCIPAL|CNAES SECUNDÁRIOS|ENDEREÇO                          |COMPLEMENTO        |BAIRRO        |CEP     |MUNICÍPIO     |UF |TELEFONE|EMAIL           

### 📊 Estatísticas Gerais

#### Listar UFs distintas

In [111]:
df.select("uf").distinct().show()

+---+
| uf|
+---+
| SP|
| MG|
| RJ|
| PR|
| RS|
| SC|
| BA|
| GO|
| PE|
| ES|
| PA|
| CE|
| MT|
| MS|
| MA|
| DF|
| PI|
| AM|
| PB|
| AL|
+---+
only showing top 20 rows


#### Contagem de empresas por UF

In [ ]:
df_contagem_uf_empresas = \
    df.groupBy("uf") \
    .agg(
        f.count("*").alias("Qtd Empresas")
    ).orderBy("Qtd Empresas", ascending=False)

df_contagem_uf_empresas.show()

+---+------------+
| uf|Qtd Empresas|
+---+------------+
| SP|     1379898|
| MG|      513667|
| RJ|      382888|
| RS|      344456|
| PR|      326165|
| BA|      237630|
| SC|      231308|
| GO|      169298|
| PE|      141706|
| CE|      134480|
| ES|       96212|
| PA|       93181|
| MT|       91885|
| DF|       79580|
| MA|       72871|
| MS|       63846|
| PB|       59851|
| RN|       53391|
| AM|       51203|
| AL|       45296|
+---+------------+
only showing top 20 rows


#### Decrição estatística

In [116]:
# Para mais de uma coluna -> df.describe(["col1", "col2"]).show()
df.describe("capital_social").show()

+-------+-------------------+
|summary|     capital_social|
+-------+-------------------+
|  count|             486655|
|   mean|    11290841.310325|
| stddev|7.893831372139556E8|
|    min|               0.00|
|    max|    500000352182.73|
+-------+-------------------+



### 🔍 3. Consultas Relevantes para Insights

#### Top 10 municípios com mais empresas

In [122]:
df_municipio = df
df_municipio_por_uf = df_municipio.groupBy("municipio", "uf").agg(
    f.count("*").alias("Qtd Municipios"),
).orderBy("Qtd Municipios", ascending=False)
df_municipio_por_uf.show(10)

+--------------+---+--------------+
|     municipio| uf|Qtd Municipios|
+--------------+---+--------------+
|     SAO PAULO| SP|        430363|
|RIO DE JANEIRO| RJ|        168441|
|BELO HORIZONTE| MG|         91174|
|      BRASILIA| DF|         79580|
|      CURITIBA| PR|         75285|
|      SALVADOR| BA|         66713|
|  PORTO ALEGRE| RS|         62352|
|     FORTALEZA| CE|         62262|
|       GOIANIA| GO|         51450|
|        RECIFE| PE|         42108|
+--------------+---+--------------+
only showing top 10 rows


#### Distribuição por Natureza Jurídica

In [ ]:
df_natureza_juridica = df


## Delta Table - Controle de versões

In [ ]:
deltaTable.history().show(vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 5                                                                                                                                                                                                                                                      

## Finalizar spark

In [160]:
spark.stop()